## Import Libraries

In [87]:
import os
import sys


## Read The PDF

In [88]:

data='data.pdf'


In [89]:
data

'data.pdf'

## Extract PDF data

In [90]:
import PyPDF2
from docx import Document

In [91]:
def extract_text_pdf(pdf):
    text = ""
    reader = PyPDF2.PdfReader(pdf)
    for page in reader.pages:
        text += page.extract_text() or ""
    return text


In [92]:
def extract_text_docx(docx):
    doc = Document(docx)
    text = ""
    for paragraph in doc.paragraphs:
        text += paragraph.text + "\n"
    return text

In [93]:
def extract(doc):
    _, file_extension = doc.split('.')
    print(file_extension)
    if file_extension == 'pdf':
        return extract_text_pdf(doc)
    elif file_extension == 'docx':
        return extract_text_docx(doc)
    else:
        raise ValueError("Unsupported file format")

In [94]:
text_data=extract(data)

pdf


In [95]:
#print(text_data)   un comment and run this to view pdf in text fromat

## Divide text data into Chunks

In [96]:
# Code here
from langchain.text_splitter import CharacterTextSplitter

In [97]:
text_splitter=CharacterTextSplitter(
    separator="\n",
    chunk_size=1300,
    chunk_overlap=300,
    length_function=len,
)

''' important, GPT-3.5 has a maximum token limit of 4096 tokens per input sequence. This means that any 
 input longer than 4096 tokens would need to be split into multiple segments for processing. '''

' important, GPT-3.5 has a maximum token limit of 4096 tokens per input sequence. This means that any \n input longer than 4096 tokens would need to be split into multiple segments for processing. '

In [98]:
chunks=text_splitter.split_text(text_data)

In [99]:
len(chunks)

61

In [100]:
print(chunks[60])

(a) A minimum  GPA  of 3.30  for Honours  Degree  Course  Units,  
(b) A minimum  GPA  of 3.30  for all course  units,  and 
(c) Grades  of A- or better  for Honours  Degree  Course  Units  aggregating  to a mini - 
mum of 12 credits  23  (iii) Second  Class  (Lower  Division)  
(a) A minimum  GPA  of 3.00  for Honours  Degree  Course  Units,  and 
(b) A minimum  GPA  of 3.00  for all course  units


## Text Embedding

In [111]:
import os
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())
api_key=os.environ.get("OPEN_AI_KEY")
embedding=OpenAIEmbeddings(openai_api_key=api_key)

In [108]:
db=FAISS.from_texts(texts=chunks,embedding=embedding)

## Similarity Check

In [116]:
query = "what are the Minimum Requirement for the Completion of the Bachelor of Computer Science Honours Degree ?"
docs = db.similarity_search(query)

In [121]:
len(docs)

4

In [120]:
print(docs[0])

page_content='(i) a minimum  Overall  Average  of 3.00  and 
(ii) grades  of B or better  in Course  Units  aggregating  to a minimum  of 40 credits.  
 
Other  Requirements  
In addition to the examination criteria stated above, the candidate have to pass in English 
proficiency LevelI and Level II examinations in order to release the final results of the 
Bachelor of Computer Science degree.  
 
1.9.4  Minimum Requirement for the Completion of the Bachelor of Computer 
Science (Honours) Degree  
A student shall be deemed to have passed the Bachelor of Computer Sci ence (Honours) 
Degree Examination if he/she has obtained following:  
(a) A minimum  of 120 credits  with  at least  30 credits  from  the fourth  academic  year  of 
Bachelor of Computer Science (Honours) degree course units,  
(b) An overall  minimum  Grade  Point  Average  (GPA)  of 2.5 for all course  units  followed 
throughout the four academic years of the degree programme and  
(c) A grade  not inferior  to C  for 

## Similarity Check by vector


In [122]:
embedding_vector = embedding.embed_query(query)
docs = db.similarity_search_by_vector(embedding_vector)
print(docs[0].page_content)

(i) a minimum  Overall  Average  of 3.00  and 
(ii) grades  of B or better  in Course  Units  aggregating  to a minimum  of 40 credits.  
 
Other  Requirements  
In addition to the examination criteria stated above, the candidate have to pass in English 
proficiency LevelI and Level II examinations in order to release the final results of the 
Bachelor of Computer Science degree.  
 
1.9.4  Minimum Requirement for the Completion of the Bachelor of Computer 
Science (Honours) Degree  
A student shall be deemed to have passed the Bachelor of Computer Sci ence (Honours) 
Degree Examination if he/she has obtained following:  
(a) A minimum  of 120 credits  with  at least  30 credits  from  the fourth  academic  year  of 
Bachelor of Computer Science (Honours) degree course units,  
(b) An overall  minimum  Grade  Point  Average  (GPA)  of 2.5 for all course  units  followed 
throughout the four academic years of the degree programme and  
(c) A grade  not inferior  to C  for the fourth yea